# CLASE 1.4: Un repaso (consciente) de cálculo diferencial.
---

## Introducción.
Muchos algoritmos de machine learning buscan la **optimización** de una **función objetivo** con respecto a un conjunto de parámetros deseados para un determinado modelo, los cuales controlan qué tan bien dicho modelo explica las observaciones representadas por la data a nuestra disposición. La búsqueda de parámetros óptimos puede entonces plantearse como un **problema de optimización**. Algunos ejemplos de tales problemas son los siguientes:

**(I) Regresión lineal:** Donde el problema radica en encontrar el mejor *hiperplano* que se ajusta a un determinado conjunto de observaciones, representadas en la práctica como puntos en el espacio $\mathbb{R}^{n}$. En general, estos problemas se plantean en la forma de ajustes de hiperplanos cuyas ecuaciones son del tipo $y=A_{1}x_{1}+A_{2}x_{2}+\dots +A_{n}x_{n}$, donde deseamos determinar los **coeficientes de regresión** $A_{1},\dots ,A_{n}$ tales que se maximice un determinado valor, llamado **valor de verosimilitud**. En la segunda parte de estos apuntes, estudiaremos en detalle la construcción de este tipo de modelos y su implementación en Python, usando fundamentalmente la librería <font color="mediumorchid">Scikit-Learn</font>.

**(II) Auto-encoders basados en redes neuronales para la reducción de la dimensión en conjuntos de datos:** En un problema de este tipo, existen dos conjuntos de parámetros de interés en nuestros modelos, conocidos como **sesgos** y **ponderadores**, donde minimizamos un **error de reconstrucción** mediante una aplicación repetida de la **regla de la cadena**, la que, como suele verse en cursos de cálculo, se trata de una metodología que nos permite diferenciar funciones compuestas por otras funciones. Los auto-encoders son modelos de alta complejidad que están fuera del alcance de estos apuntes.

**(III) Modelos de mixtura Gaussiana:** En estos problemas deseamos modelar **distribuciones de probabilidad** asociadas a un conjunto de datos mediante la optimización de ciertos parámetros que explican la geometría intrínseca a tales distribuciones, denominados **parámetros de forma** y **escala**, suponiendo que éstas distribuciones pueden aproximarse mediante sumas o **mixturas** de subconjuntos de datos normalmente distribuidos (llamados usualmente **componentes de mixtura**).

Algunos de los conceptos anteriores se ilustran en la Fig. (4.1).

<p style="text-align: center;"><img src="figures/fig_4_1.png" width="900"></p>
<p style="text-align: center;">Fig. (4.1): (a) En un problema de regresión lineal, queremos encontrar parámetros tales que la curva (en la imagen, una recta) explique adecuadamente las observaciones representadas por puntos en el gráfico; (b) En un problema de estimación de densidad mediante un modelo de mixtura Gaussiana, queremos encontrar las medias y covarianzas tales que podemos explicar adecuadamente las observaciones (puntos). </p>

Un elemento central de esta sección es el concepto de función. Una **función** $f$ es una entidad que relaciona dos cantidades mediante una correspondencia bien definida. En el resto de estos apuntes, tales cantidades serán típicamente entradas vectoriales del tipo $\mathbf{x}\in \mathbb{R}^{d}$ y salidas unidimensionales o **valores objetivo** (lo que escribimos como $f(\mathbf{x})$), que asumiremos como pertenencientes a $\mathbb{R}$, salvo que indiquemos específicamente lo contrario.

En nuestra conceptualización de función, $\mathbb{R}^{d}$ corresponde al **dominio** de $f$, mientras que los valores $f(\mathbf{x})$ corresponden a las **imágenes** de $\mathbf{x}$ bajo $f$. El conjunto de todas las posibles imágenes conforman el **codominio** de $f$, siendo todos estos conceptos completamente análogos a los vistos para las transformaciones lineales estudiadas en la [Clase 1.1](https://github.com/rquezadac/udd_data_science_lectures/blob/main/PARTE%20I%20-%20Fundamentos%20matem%C3%A1ticos%20elementales/clase_1_1.ipynb). Con frecuencia, escribimos

$$\begin{array}{ll}f:&\mathbb{R}^{d} \longrightarrow \mathbb{R} \\ &\mathbf{x} \longmapsto f\left( \mathbf{x} \right)  \end{array}$$
<p style="text-align: right;">$(4.1)$</p>

para especificar una función.

En esta sección discutiremos fundamentalmente cómo computar **gradientes de funciones**, lo que frecuentemente resulta esencial para **facilitar el proceso de aprendizaje de los algoritmos de machine learning**, debido a que los vectores gradientes suelen tener direcciones coincidentes con la posición de mínimos locales en hipersuperficies (superficies en $\mathbb{R}^{d}$) que representan una determinada **función de costo** a optimizar en la construcción de un modelo. Por lo tanto, el cálculo diferencial es una de las herramientas matemáticas fundamentales que necesitamos para entender, al nivel más elemental, lo que son los algoritmos de machine learning.

## Diferenciación de funciones de una variable.
A continuación, revisaremos brevemente el concepto de diferenciación de funciones de una variable real, el cual se suele aprender el primer curso de cálculo. Partiremos entonces con funciones del tipo $f:I\subseteq \mathbb{R} \longrightarrow \mathbb{R}$, donde $I=(a,b)$ es un intervalo abierto en $\mathbb{R}$, para las cuales estableceremos la siguiente definición.

**<font color='blue'>Definición 4.1 – Cociente de diferencias:</font>** El **cociente de diferencias**

$$\frac{\delta y}{\delta x} :=\frac{f\left( x+\delta x\right)  -f\left( x\right)  }{\delta x}$$
<p style="text-align: right;">$(4.2)$</p>

Nos permite calcular la pendiente de la recta secante que pasa por los puntos $(x,f(x))$ y $(x+\delta x,f(x+\delta x))$, por los cuales también pasa el gráfico de $f$. Esta situación se esquematiza en la Fig. (4.2).

<p style="text-align: center;"><img src="figures/fig_4_2.png" width="300"></p>
<p style="text-align: center;">Fig. (4.2): Situación esquemática que representa a una recta secante que pasa por los puntos $(x,f(x))$ y $(x+\delta x,f(x+\delta x))$ del gráfico de $f$, y los valores asociados al cociente de diferencias $\frac{\delta y}{\delta x}$</p>

El cociente de diferencias (4.2) también puede ser considerado como la pendiente promedio de $f$ entre $x$ y $x+\delta x$ siempre que $f$ sea una función lineal. En el límite, cuando $\delta x \rightarrow 0$, obtenemos la tangente de $f$ en $x$, siempre que $f$ cumpla con ser continua en dicho punto. El valor del cociente de diferencias en este límite se conoce como **derivada** de $f$ en $x$. Tiene sentido, por tanto, la siguiente definición.

**<font color='blue'>Definición 4.2 – Derivada de una función en un punto:</font>** Sea $f:I\subseteq \mathbb{R}\longrightarrow \mathbb{R}$ una función definida explícitamente en el intervalo abierto $I$ de $\mathbb{R}$ (y que escribimos como $y=f(x)$). Sea $x_{0}\in I$ y $h>0$. Definimos entonces la **derivada de $f$ en el punto $x_{0}$** como

$$\frac{df}{dx} \left( x_{0}\right)  :=\lim_{h\rightarrow 0} \frac{f\left( x_{0}+h\right)  -f\left( x_{0}\right)  }{h}$$
<p style="text-align: right;">$(4.3)$</p>

La **notación** para las derivadas es diversa. De esta manera, la derivada de $f$ den el punto $x_{0}$ puede denotarse como $\frac{df}{dx} \left( x_{0}\right)$, donde $\frac{d}{dx}$ es el llamado **operador diferencial**, y que se lee *"derivada con respecto a la variable $x$"*. Otras notaciones incluyen $f^{\prime }\left( x_{0}\right)$ e incluso $Df\left( x_{0}\right)$. En estos apuntes, usaremos las dos primeras de manera intercambiable.

**<font color='blue'>Definición 4.3 – Derivada de una función en un conjunto:</font>** Sea $f:I\subseteq \mathbb{R}\longrightarrow \mathbb{R}$ una función definida explícitamente en el intervalo abierto $I$ de $\mathbb{R}$. Definimos la derivada de $f$ en el conjunto $U\subseteq I$ como

$$\frac{df}{dx} =\lim_{h\rightarrow 0} \frac{f\left( x+h\right)  -f\left( x\right)  }{h}$$
<p style="text-align: right;">$(4.4)$</p>

para todo $x\in U$.

**<font color='blue'>Definición 4.4 – Diferenciabilidad:</font>** Sea $f:I\subseteq \mathbb{R}\longrightarrow \mathbb{R}$ una función definida explícitamente en el intervalo abierto $I$ de $\mathbb{R}$. Si la derivada de $f$ existe para $x_{0}\in I$, entonces diremos que $f$ es **diferenciable** en el punto $x_{0}$. Por otro lado, si $f$ es diferenciable en todos los puntos de $I$, diremos que $f$ es diferenciable en todo $I$.

**Ejemplo 4.1:** Vamos a usar la definición de derivada (ecuación (4.4)) para calcular la derivada de la función $y=x^{n}$ para $n\in \mathbb{N}$ y todo $x\in \mathbb{R}$. En efecto,

$$\begin{array}{lll}\displaystyle \frac{dy}{dx} &=&\displaystyle \lim_{h\rightarrow 0} \displaystyle \frac{\left( x+h\right)^{n}  -x^{n}}{h} \  \left( \mathrm{aplicamos} \  \mathrm{la} \  \mathrm{formula} \  \mathrm{del} \  \mathrm{binomio} \  \mathrm{de} \  \mathrm{Newton} \right)  \\ &=&\displaystyle \lim_{h\rightarrow 0} \displaystyle \frac{\displaystyle \sum\nolimits^{n}_{k=0} \displaystyle \binom{n}{k} x^{n-k}h^{k}-x^{n}}{h} \  \left( \mathrm{recordemos} \  \mathrm{que} \  \displaystyle \binom{n}{k} =\displaystyle \frac{n!}{\left( n-k\right)  !k!} \right)  \\ &=&\displaystyle \lim_{h\rightarrow 0} \displaystyle \frac{x^{n}+nx^{n-1}h+\overbrace{\frac{n\left( n-1\right)  }{2!} x^{n-2}h^{2}+\cdots +h^{n}}^{\rightarrow 0\  \mathrm{cuando} \  h\rightarrow 0} -x^{n}}{h} \\ &=&\displaystyle \frac{x^{n}+nx^{n-1}h-x^{n}}{h} =nx^{n-1}\end{array}$$
<p style="text-align: right;">$(4.5)$</p>
◼︎

**Ejemplo 4.2:** Vamos a usar nuevamente la definición de derivada (ecuación (4.4)) para calcular la derivada de la función $y=\mathrm{sen}(x)$. En efecto,

$$\begin{array}{lll}\displaystyle \frac{dy}{dx} &=&\displaystyle \lim_{h\rightarrow 0} \displaystyle \frac{\mathrm{sen} \left( x+h\right)  -\mathrm{sen} \left( x\right)  }{h} \\ &=&\displaystyle \lim_{h\rightarrow 0} \displaystyle \frac{\mathrm{sen} \left( x\right)  \cos \left( h\right)  -\cos \left( x\right)  \mathrm{sen} \left( h\right)  -\mathrm{sen} \left( x\right)  }{h} \\ &=&\mathrm{sen} \left( x\right)  \underbrace{\displaystyle \lim_{h\rightarrow 0} \left( \displaystyle \frac{\cos \left( h\right)  -1}{h} \right)  }_{=0\  \mathrm{cuando} \  h\rightarrow 0} +\cos \left( x\right)  \underbrace{\displaystyle \lim_{h\rightarrow 0} \left( \displaystyle \frac{\mathrm{sen} \left( h\right)  }{h} \right)  }_{=1\  \mathrm{cuando} \  h\rightarrow 0} \\ &=&\cos \left( x\right)  \end{array}$$
<p style="text-align: right;">$(4.6)$</p>
◼︎

### Álgebra de derivadas.
A continuación, revisaremos brevemente algunas reglas básicas que nos permitirán calcular las derivadas de funciones compuestas por funciones más simples: Sumas, multiplicaciones, cocientes y composiciones.

**<font color='crimson'>Teorema 4.1:</font>** *Sean $f,g:I\subseteq \mathbb{R} \longrightarrow \mathbb{R}$ funciones diferenciables en el intervalo abierto $I$ de $\mathbb{R}$. Entonces se tiene que*

- **(T1):** *Derivada de una suma: $\displaystyle \frac{d}{dx} \left( f\left( x\right)  \pm g\left( x\right)  \right)  =\frac{df}{dx} \left( x\right)  \pm \frac{dg}{dx} \left( x\right)$*.
- **(T2):** *Derivada de un producto: $\displaystyle \frac{d}{dx} \left( f\left( x\right)  g\left( x\right)  \right)  =g\left( x\right)  \frac{df}{dx} \left( x\right)  +f\left( x\right)  \frac{dg}{dx} \left( x\right)$*.
- **(T3):** *Derivada de un cociente: $\displaystyle \frac{d}{dx} \left( \frac{f\left( x\right)  }{g\left( x\right)  } \right)  =\frac{g\left( x\right)  \frac{df}{dx} \left( x\right)  +f\left( x\right)  \frac{dg}{dx} \left( x\right)  }{\left[ g\left( x\right)  \right]^{2}  }$*.

◆

Un caso particular de (T2) se da cuando $g(x)=c$ para todo $x$ en el dominio de $g$, donde $c$ es una constante. Como la derivada de una función constante de este tipo es nula para todo $x$, entonces aplicando la fórmula de la derivada de un producto a $cf(x)$ para $c\in \mathbb{R}$ obtenemos

$$\frac{d}{dx} \left( cf\left( x\right)  \right)  =c\frac{df}{dx} \left( x\right)  =cf^{\prime }\left( x\right)$$
<p style="text-align: right;">$(4.7)$</p>

Lo que significa que si calculamos la derivada de una función multiplicada por una constante, entonces el resultado es igual a dicha constante multiplicada por la derivada de la función. Al combinar esto con (T1), podemos obtener la fórmula general

$$\frac{d}{dx} \left( c_{1}f\left( x\right)  +c_{2}g\left( x\right)  \right)  =c_{1}\frac{df}{dx} \left( x\right)  +c_{2}\frac{dg}{dx} \left( x\right)$$
<p style="text-align: right;">$(4.8)$</p>

para $c_{1}\wedge c_{2}\in \mathbb{R}$. Notemos que, si llamamos $D$ al operador diferencial, las fórmulas (4.7) y (T1) nos muestran que, para cualquier constante $c\in \mathbb{R}$ y funciones diferenciables $f,g$, se tiene que $D(f+g)=D(f)+D(g)$ y $D(cf)=cD(f)$. Por lo tanto, **este operador (y, por extensión, la derivada) se trata de una transformación lineal** definida sobre el cuerpo $\mathbb{R}$ y el espacio vectorial definido por todas las funciones diferenciables sobre el intervalo abierto $I=(a,b)\in \mathbb{R}$.

Dado que un requisito esencial de esta asignatura es precisamente el curso de cálculo, no ahondaremos mucho más en el concepto de derivada ni tampoco nos abstraeremos demasiado por el momento. Asumiremos pleno conocimiento de las derivadas de funciones trascendentes, con lo cual, comenzaremos a trabajar algunos ejemplos sencillos.

**Ejemplo 4.3:** Calcularemos la derivada de la función $f$ definida como

$$f\left( x\right)  =\frac{a\cos \left( x\right)  +b\ln \left( x\right)  }{a^{2}+x^{2}}$$
<p style="text-align: right;">$(4.9)$</p>

Vemos que la función $f$ no es mmás que el resultado de una combinación de operaciones algebraicas entre varias funciones más sencillas. Si escribimos $f(x)=f_{1}(x)/f_{2}(x)$, entonces $f_{1}(x)=a\cos \left( x\right)  +b\ln \left( x\right)$ y $f_{2}(x)=a^{2}+x^{2}$. Luego debemos aplicar la fórmula de la derivada de un cociente, con lo cual

$$\begin{array}{lll}f\left( x\right)  =\displaystyle \frac{a\cos \left( x\right)  +b\ln \left( x\right)  }{a^{2}+x^{2}} &\Longrightarrow &f^{\prime }\left( x\right)  =\displaystyle \frac{f_{2}\left( x\right)  f^{\prime }_{1}\left( x\right)  -f_{1}(x)f^{\prime }_{2}\left( x\right)  }{\left[ f_{2}\left( x\right)  \right]^{2}  } \\ &\Longrightarrow &f^{\prime }\left( x\right)  =\displaystyle \frac{\left( a^{2}+x^{2}\right)  \left( -a\  \mathrm{sen} \left( x\right)  +\frac{b}{x} \right)  -\left( a\cos \left( x\right)  +b\ln \left( x\right)  \right)  \left( 2x\right)  }{\left( a^{2}+x^{2}\right)^{2}  } \\ &\Longrightarrow &f^{\prime }\left( x\right)  =\displaystyle \frac{\left( a^{2}+x^{2}\right)  \left( \frac{b}{x} -a\  \mathrm{sen} \left( x\right)  \right)  -2x\left( a\cos \left( x\right)  +b\ln \left( x\right)  \right)  }{\left( a^{2}+x^{2}\right)^{2}  } \\ &\Longrightarrow &f^{\prime }\left( x\right)  =\displaystyle \frac{\frac{a^{2}+x^{2}}{x} \left( b-ax\  \mathrm{sen} \left( x\right)  \right)  -2x\left( a\cos \left( x\right)  +b\ln \left( x\right)  \right)  }{\left( a^{2}+x^{2}\right)^{2}  } \end{array}$$
<p style="text-align: right;">$(4.10)$</p>
◼︎

### Regla de la cadena.
Con las fórmulas de derivación dadas hasta ahora, se pueden calcular derivadas de funciones $f$ para las cuales $f(x)$ es una suma finita de cocientes o productos de constantes multiplicadas por funciones cuyos argumentos sea únicamente la variable $x$. Sin embargo, existen funciones que están compuestas por otras funciones que no son posibles de derivar con las reglas de derivación estudiadas previamente, tales como $f(x)=\mathrm{sen}(x^{2})$ o $g\left( x\right)  =\arctan \left( x^{2}-\ln \left( 1+x\right)  \right)$. Tales funciones se denominan **composiciones**, y pueden escribirse como $f\left( g\left( x\right)  \right)$.

Para derivar una **composición de funciones** del tipo $y=f\left( g\left( x\right)  \right)$, se hace necesario contar con una regla de derivación adicional. Dicha regla existe y permite expresar la derivada $dy/dx$ en función de las derivadas de $f$ y $g$.

**<font color='crimson'>Teorema 4.2 – Regla de la cadena:</font>** *Sea $f$ la función resultante de la composición de dos funciones $u$ y $v$, de manera que $f$ puede definirse explícitamente  por la fórmula $f\left( x\right)  =u\left( v\left( x\right)  \right)$. Si existen las derivadas $u'(x)$ y $v'(x)$ en los puntos donde $f$ está definida, para $y=v(x)$, entonces la derivada $f'(x)$ existe y está dada por la fórmula*

$$f^{\prime }\left( x\right)  =u^{\prime }\left( v\left( x\right)  \right)  v^{\prime }\left( x\right)$$
<p style="text-align: right;">$(4.11)$</p>
◆

**Ejemplo 4.4:** Vamos a calcular al derivada de la función $f$ definida como

$$f\left( x\right)  =\ln \left( \frac{5-2^{x}}{3+\cos \left( x\right)  } \right)$$
<p style="text-align: right;">$(4.12)$</p>

Vemos que $f$ es una composición de dos funciones $u$ y $v$ definidas como $u(x)=\ln (x)$ y $v\left( x\right)  =\frac{5-2^{x}}{3+\cos \left( x\right)  }$. Por lo tanto, debemos aplicar la regla de la cadena para calcular la derivada $f'(x)$ en función de las derivadas $u(x)$ y $v(x)$ para $y=v(x)$. De esta manera, tenemos que

$$\frac{du}{dx} =\frac{1}{y} =\frac{1}{\left( \frac{5-2^{x}}{3+\cos \left( x\right)  } \right)  } \  \wedge \  \frac{dv}{dx} =\frac{\left( 3+\cos \left( x\right)  \right)  \left( -2^{x}\ln \left( 2\right)  \right)  -\left( 5-2^{x}\right)  \left( -\mathrm{sen} \left( x\right)  \right)  }{\left( 3+\cos \left( x\right)  \right)^{2}  }$$
<p style="text-align: right;">$(4.13)$</p>

Así que,

$$f^{\prime }\left( x\right)  =u^{\prime }\left( y\right)  v^{\prime }\left( x\right)  =\frac{2^{x}\ln \left( 2\right)  \left( \cos \left( x\right)  -3\right)  +\left( 5-2^{x}\right)  \mathrm{sen} \left( x\right)  }{\left( 5-2^{x}\right)  \left( 3+\cos \left( x\right)  \right)  }$$
<p style="text-align: right;">$(4.14)$</p>
◼︎

### Series de Taylor.
La **serie de Taylor** es una representación de una función $f$ por medio de una serie infinita cuyos términos se determinan mediante las derivadas sucesivas de $f$ evaluadas en un punto dentro del dominio de la función. Estas series son muy utilizadas en el campo del análisis numérico para la aproximación de funciones por medio de sumas finitas cortando la serie en un término determinado.

**<font color='blue'>Definición 4.5 – Polinomio de Taylor:</font>** Sea $f:I\subseteq \mathbb{R} \longrightarrow \mathbb{R}$ una función definida en el intervalo abierto $I$ de $\mathbb{R}$, $n$ veces diferenciable en $I$ (es decir, sus primeras $n$ derivadas existen en $I$, lo que equivale nombrar a $f$ como función de clase $C^{k}$ para $k\leq n$). Para $x_{0}\in I$, definimos el **polinomio de Taylor** de orden $n$ de $f$ como

$$T_{n}\left( x\right)  :=\sum^{n}_{k=0} \frac{f^{\left( k\right)  }\left( x_{0}\right)  }{k!} \left( x-x_{0}\right)^{k}$$
<p style="text-align: right;">$(4.15)$</p>

Donde $f^{\left( k\right)  }\left( x_{0}\right)$ denota la $k$-ésima derivada de $f$ en el punto $x_{0}$.

**<font color='blue'>Definición 4.6 – Serie de Taylor:</font>** Sea $f$ una función infinitamente diferenciable en un intervalo $I\in \mathbb{R}$ (lo que solemos denotar como función de clase $C^{\infty}$ en $I$). Definimos la **serie de Taylor** de $f$ centrada en el punto $x_{0}\in I$ como

$$T_{\infty }\left( x\right)  :=\sum^{+\infty }_{k=0} \frac{f^{\left( k\right)  }\left( x_{0}\right)  }{k!} \left( x-x_{0}\right)^{k}$$
<p style="text-align: right;">$(4.16)$</p>